In [59]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
import json
import os
import time

from faker import Faker
import babel
from babel.dates import format_date

import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM, Embedding

import tensorflow.contrib.legacy_seq2seq as seq2seq
from utilities import show_graph

from sklearn.model_selection import train_test_split
print('Imports successful')

Imports successful


In [90]:
fake = Faker()
fake.seed(42)
random.seed(42)

FORMATS = ['short',
           'medium',
           'long',
           'full',
           'd MMM YYY',
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY',
           ]

# change this if you want it to work with only a single language
LOCALES = babel.localedata.locale_identifiers()
LOCALES = [lang for lang in LOCALES if 'en' in str(lang)]
print(LOCALES)

['en_PK', 'en_GG', 'en_TZ', 'en_VU', 'en_UG', 'en_US_POSIX', 'en_IM', 'en_GD', 'en_CC', 'en_SD', 'en_TC', 'en_NR', 'en_DM', 'en_SX', 'en_AS', 'en_PR', 'en_NU', 'en_FJ', 'en_FK', 'en_MY', 'en_SB', 'en_BE', 'en_MO', 'en_IE', 'en', 'en_BS', 'en_LR', 'en_ZW', 'en_SS', 'en_FI', 'en_MG', 'en_KE', 'en_ZA', 'en_KN', 'en_CX', 'en_TK', 'en_GB', 'en_PG', 'en_IL', 'en_DE', 'en_NA', 'en_GH', 'en_LS', 'en_IN', 'en_HK', 'en_RW', 'en_AU', 'en_PW', 'en_VI', 'en_BM', 'en_GI', 'en_NG', 'en_BB', 'en_SZ', 'en_KI', 'en_UM', 'en_ER', 'en_AI', 'en_NF', 'en_VG', 'en_CK', 'en_BI', 'en_GY', 'en_BZ', 'en_MW', 'en_DK', 'en_KY', 'en_VC', 'en_CA', 'en_WS', 'en_AG', 'en_GU', 'en_JM', 'en_IO', 'en_PH', 'en_150', 'en_NL', 'en_BW', 'en_SL', 'en_MH', 'en_DG', 'en_CY', 'en_TO', 'en_CH', 'en_TT', 'en_MT', 'en_ZM', 'en_SH', 'en_FM', 'en_SC', 'en_AT', 'en_TV', 'en_PN', 'en_SE', 'en_SI', 'en_MU', 'en_MS', 'en_CM', 'en_GM', 'en_001', 'en_MP', 'en_LC', 'en_US', 'en_SG', 'en_NZ', 'en_JE']


In [91]:
def create_date():
    """
        Creates some fake dates 
        :returns: tuple containing 
                  1. human formatted string
                  2. machine formatted string
                  3. date object.
    """
    dt = fake.date_object()

    # wrapping this in a try catch because
    # the locale 'vo' and format 'full' will fail
    try:
        human = format_date(dt,
                            format=random.choice(FORMATS),
                            locale=random.choice(LOCALES))

        case_change = random.randint(0,3) # 1/2 chance of case change
        if case_change == 1:
            human = human.upper()
        elif case_change == 2:
            human = human.lower()

        machine = dt.isoformat()
    except AttributeError as e:
        return None, None, None

    return human, machine #, dt

data = [create_date() for _ in range(50000)]

In [92]:
data[:5]

[(u'04, DEC 2000', '2000-12-04'),
 (u'thursday, 18 march 1971', '1971-03-18'),
 (u'APRIL 21, 1983', '1983-04-21'),
 (u'17/10/1980', '1980-10-17'),
 (u'14/08/05', '2005-08-14')]

In [96]:
x = [x for x, y in data]
y = [y for x, y in data]

u_characters = set(' '.join(x))
char2numX = dict(zip(u_characters, range(len(u_characters))))

u_characters = set(' '.join(y))
char2numY = dict(zip(u_characters, range(len(u_characters))))

In [97]:
char2numX['<PAD>'] = len(char2numX)
num2charX = dict(zip(char2numX.values(), char2numX.keys()))
max_len = max([len(date) for date in x])

print('checking x', x, char2numX)
x = [[char2numX['<PAD>']]*(max_len - len(date)) +[char2numX[x_] for x_ in date] for date in x]
print('checking x', x)
print(''.join([num2charX[x_] for x_ in x[4]]))
x = np.array(x)

('checking x', u'01.11.05', {u' ': 4, u'.': 2, u'1': 0, u'0': 1, '<PAD>': 5, u'5': 3})
('checking x', [[1], [0], [2], [0], [0], [2], [1], [3]])
1


In [81]:
char2numY['<GO>'] = len(char2numY)
num2charY = dict(zip(char2numY.values(), char2numY.keys()))

y = [[char2numY['<GO>']] + [char2numY[y_] for y_ in date] for date in y]
print(''.join([num2charY[y_] for y_ in y[4]]))
y = np.array(y)

<GO>2005-08-14


In [82]:
x_seq_length = len(x[0])
y_seq_length = len(y[0])- 1 # without last <END>

In [83]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
#     from IPython.core.debugger import Tracer; Tracer()()
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [84]:
epochs = 2
batch_size = 128
nodes = 32
embed_size = 10

tf.reset_default_graph()
sess = tf.InteractiveSession()

# Tensor where we will feed the data into graph
inputs = tf.placeholder(tf.int32, (None, x_seq_length), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# Embedding layers
input_embedding = tf.Variable(tf.random_uniform((len(char2numX), embed_size), -1.0, 1.0), name='enc_embedding')
# TODO: create the variable output embedding
output_embedding = tf.Variable(tf.random_uniform((len(char2numY), embed_size), -1.0, 1.0), name='dev_embedding')
# TODO: Use tf.nn.embedding_lookup to complete the next two lines
date_input_embed = tf.nn.embedding_lookup(input_embedding, inputs)
date_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)

with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=date_input_embed, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    # TODO: create the decoder LSTMs, this is very similar to the above
    # you will need to set initial_state=last_state from the encoder
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=date_output_embed, initial_state=last_state)
#connect outputs to 
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(char2numY), activation_fn=None) 
with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

In [85]:
dec_outputs.get_shape().as_list()

[None, None, 32]

In [86]:
last_state[0].get_shape().as_list()

[None, 32]

In [87]:
inputs.get_shape().as_list()

[None, 1]

In [88]:
date_input_embed.get_shape().as_list()

[None, 1, 10]

In [89]:
show_graph(tf.get_default_graph().as_graph_def())

In [100]:
x = x.reshape(8, 1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

[[1]
 [0]
 [2]
 [0]
 [0]
 [2]
 [1]
 [3]]


ValueError: Found input variables with inconsistent numbers of samples: [8, 50000]